## Pandas

In [ ]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.simplefilter('ignore', UserWarning)

In [ ]:
df = pd.read_csv('housing.csv')

In [ ]:
house_xl = pd.read_excel('housing.xlsx', sheet_name='housing', parse_dates=True, encoding='utf-8')

In [ ]:
df.head()

In [ ]:
df.head().T

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum(axis = 0)

In [ ]:
df.columns

In [ ]:
df['SalePrice'].describe()

In [ ]:
df['SalePrice'].hist();

In [ ]:
sns.distplot(df['SalePrice']);

In [ ]:
#correlation matrix
corrmat = df.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
#scatterplot
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df[cols], size = 2.5)
plt.show();

In [ ]:
df[cols].head()

In [ ]:
from scipy import stats
#histogram and normal probability plot
sns.distplot(df['SalePrice'], fit=stats.norm);
fig = plt.figure()
res = stats.probplot(df['SalePrice'], plot=plt)

In [ ]:
#applying log transformation
df['SalesPriceLog'] = np.log(df['SalePrice'])

In [ ]:
df['SalesPriceSquared'] = df['SalePrice'].apply(lambda x: x**2)

In [ ]:
df.head()

In [ ]:
#transformed histogram and normal probability plot
sns.distplot(df['SalesPriceLog'], fit=stats.norm);
fig = plt.figure()
res = stats.probplot(df['SalesPriceLog'], plot=plt)

In [ ]:
df.sort_values('SalePrice', ascending=False, inplace=True)

In [ ]:
df['Shift'] = df['SalePrice'].shift()

In [ ]:
df.sort_values('SalePrice', ascending=False)

In [ ]:
group = df.groupby(['YrSold'])['SalePrice'].mean()
group.sort_values(ascending=False)

In [ ]:
group_df = pd.DataFrame(group).reset_index()

In [ ]:
group_df.head()

In [ ]:
group_df.to_csv('salesmeanbyyearsold.csv', index=False)

## SQL Databases

In [ ]:
import warnings
import pyodbc, secrets
import sqlalchemy as db 
import urllib.parse
warnings.filterwarnings('ignore')

In [ ]:
params = urllib.parse.quote_plus('''DRIVER={SQL Server}; 
                                    SERVER=%s; 
                                    DATABASE=%s;
                                    Trusted_Connection=yes;''' %(secrets.server, secrets.database))

In [ ]:
engine = db.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params, max_overflow=-1)

In [ ]:
connection = engine.connect()
metadata = db.MetaData()
nola_temps = db.Table('WX_MSY_Temperature', metadata, autoload=True, autoload_with=engine)

In [ ]:
print(nola_temps.columns.keys())

In [ ]:
result_proxy = engine.execute("SELECT distinct date as DateTime, convert(date, date) as Date, datepart(hour, date) as Hour, temperature as Temperature FROM wx_msy_temperature where date >= '2018-05-01' order by 1,2")

In [ ]:
result = result_proxy.fetchall()

In [ ]:
result[:5]

In [ ]:
print(result[0].keys())

In [ ]:
temps = pd.DataFrame(result, columns=result[0].keys())

In [ ]:
temps.head()

In [ ]:
temp_group = temps.groupby(['Date'])['Temperature'].mean()
temp_group

In [ ]:
housing = db.Table('housing', metadata, autoload=True, autoload_with=engine)

In [ ]:
housing.drop(engine)

In [ ]:
group_df.to_sql('housing', con=engine, if_exists='replace', index=False, chunksize=10)